# Le parole di Sanremo

**1. Le parole più frequenti nei testi di Sanremo**

In [83]:
from bs4 import BeautifulSoup as bs
import requests
import re
from collections import Counter
import pandas as pd

- Una funzione per estrapolare il testo dalla pagina web, separare le parole e convertirle in minuscolo:

In [222]:
def song_to_words(link):
    page = bs(requests.get(link).content)
    text = re.sub('’', ' ', page.select('h2 ~ p')[0].get_text())
    Song = re.split(' ', text)
    song = [re.sub(r'\W','',word.lower()) for word in Song]
    return song

- Creo una lista con i link di ogni canzone e una lista che contenga tutte le parole:

In [223]:
page = bs(requests.get('https://www.fanpage.it/spettacolo/eventi/i-testi-e-i-significati-delle-30-canzoni-in-gara-a-sanremo-2024/').content)
songs = [x for x in set(page.select('p a')[1:])]
links = [x['href'] for x in songs]

In [227]:
words = []
for link in links:
    words += song_to_words(link)

- Creo un dataframe a partire dal dizionario Counter che conta le ricorrenze di ogni parola:

In [230]:
df = pd.DataFrame.from_dict(Counter(words), orient='index', columns=['Conteggio']).reset_index(names='Parola')

In [231]:
df.sort_values(by=['Conteggio'], ascending=False).head(20)

,Parola,Conteggio
36,non,264
0,e,248
28,che,224
9,un,200
8,di,193
5,è,179
6,la,159
143,mi,142
70,ti,133
30,il,133


- Se vogliamo considerare le parole di almeno 4 caratteri:

In [243]:
df[[len(x)>3 for x in df['Parola']]].sort_values(by=['Conteggio'],ascending=False).head(20)

,Parola,Conteggio
122,come,80
131,sono,70
118,quando,54
132,solo,54
89,anche,49
266,ancora,48
331,tutto,36
750,boom,36
97,perché,32
65,amore,31


**2. Le possibili squadre del Fantasanremo**

- Recupero le quotazioni dei singoli artisti:

In [245]:
fanta = bs(requests.get('https://tg24.sky.it/spettacolo/musica/2023/12/28/fantasanremo-quotazioni').content)

In [312]:
cantanti = [x.get_text() for x in fanta.select('div.c-article-section:nth-child(5) ul li')]

In [314]:
for i,x in enumerate(cantanti):
    if x[-10] == '.':
        cantanti[i] = cantanti[i][:-10] + ':' + cantanti[i][-9:]

In [316]:
quote = {}
for cantante in cantanti:
    nome = re.split(':', cantante)[0].strip()
    quota = int(re.split(':', cantante)[1].strip()[:2])
    quote[nome] = quota

In [317]:
quote

{'Alessandra Amoroso': 23,
 'Alfa': 20,
 'Angelina Mango': 23,
 'Annalisa': 23,
 'Big Mama': 17,
 'bnkr44': 16,
 'Clara': 16,
 'Dargen D’Amico': 22,
 'Diodato': 21,
 'Emma': 23,
 'Fiorella Mannoia': 20,
 'Fred De Palma': 18,
 'Gazzelle': 18,
 'Geolier': 22,
 'Ghali': 20,
 'Il Tre': 17,
 'Il Volo': 20,
 'Irama': 19,
 'La Sad': 19,
 'Loredana Berté': 18,
 'Mahmood': 21,
 'Maninni': 17,
 'Mr. Rain': 21,
 'Negramaro': 22,
 'Renga e Nek': 20,
 'Ricchi e Poveri': 18,
 'Rose Villain': 19,
 'Sangiovanni': 21,
 'Santi Francesi': 16,
 'The Kolors': 22}

- Calcolo tutte le possibili squadre e seleziono quelle per cui la somma dei costi sia al più 100:

In [340]:
from itertools import combinations
squadre_tot = list(combinations(list(quote.keys()), 5))

In [347]:
def somma_squadra(squadra):
    somma = 0
    for x in squadra:
        somma += quote[x]
    return somma

In [369]:
squadre_possibili = {}
for squadra in squadre_tot:
    if somma_squadra(squadra) <= 100:
        squadre_possibili[squadra] = somma_squadra(squadra)

In [370]:
len(squadre_possibili)

92337